# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment
import re

# Import dos dados

In [2]:
df = pd.read_csv("tb_2013_2015_2017_2019_municipio.csv")

dfAnosIniciais = df.copy()

dfAnosIniciaisTeste = df.copy()


# Pré-Processamento dos dados

In [3]:
# Pré-Processamento dos dados de treino e teste dos anos iniciais

# Removendo colunas que não serão utilizadas
colunas_a_serem_dropadas = [
    "nomePais", "codRegiao", "nomeRegiao", "codEstado", "codMunicipio","valor", "AFDMedioG1", "AFDMedioG2","AFDMedioG3",
         "AFDMedioG4", "AFDMedioG5","ATUMedioTotal", "ATUMedio1Serie",
         "ATUMedio2Serie", "ATUMedio3Serie", "ATUMedioNaoSeriado", "ATUMedio4Serie", "IEDEnsinoMedioNivel5", 
         "DSUMedio", "HADMedioTotal", "HADMedio1Serie", "HADMedio2Serie",
         "HADMedio3Serie", "HADMedio4Serie", "HADMedioNaoSeriado",
         "ICGNivelMedio", "IEDEnsinoMedioNivel1","IEDEnsinoMedioNivel2",
         "IEDEnsinoMedioNivel3", "IEDEnsinoMedioNivel4", "IEDEnsinoMedioNivel6",
         "TDIMedio1Serie","TDIMedio2Serie", "TDIMedio3Serie", "TDIMedio4Serie", "nomeMunicipio", "nomeEstado", "AFDInfantilG2",
        "AFDInfantilG3", "AFDInfantilG4", "AFDInfantilG5", "AFDFundamentalTotalG1", "AFDFundamentalTotalG2", "AFDFundamentalTotalG3", "AFDFundamentalTotalG4",
        "AFDFundamentalTotalG5", "InvestAlunoInfantil", "InvestAlunoMedio", "AFDInfantilG1", "ATUInfantilTotal", "ATUInfantilCreche", "ATUInfantilPreEscola",
        "ATUFundamentalTotal", "ATUFundamental1Ano", "ATUFundamental2Ano",
        "ATUFundamental3Ano", "ATUFundamental4Ano", "ATUFundamental5Ano",
        "ATUFundamentalMultietapa", "DSUInfantilTotal", "DSUInfantilCreche",
        "DSUInfantilPreEscola", "DSUFundamentalTotal", "HADInfantilTotal", 
        "HADInfantilCreche", "HADInfantilPreEscola", "HADFundamentalTotal",
        "HADFundamental1Ano", "HADFundamental2Ano", "HADFundamental3Ano", 
        "HADFundamental4Ano", "HADFundamental5Ano", "IEDFundamentalTotalNivel1", "IEDFundamentalTotalNivel2", 
        "IEDFundamentalTotalNivel3", "IEDFundamentalTotalNivel4",
        "IEDFundamentalTotalNivel5", "IEDFundamentalTotalNivel6",
        "IRDMediaTotal", "TDIFundamental1Ano", "TDIFundamental2Ano",
        "TDIFundamental3Ano", "TDIFundamental4Ano", "TDIFundamental5Ano",
        "TDIFundamentalTotal", "anosFinais", "AFDFundamentalAnosFinaisG1", "AFDFundamentalAnosFinaisG2",
        "AFDFundamentalAnosFinaisG3", "AFDFundamentalAnosFinaisG4", "AFDFundamentalAnosFinaisG5",
        "ATUFundamentalAnosFinais", "ATUFundamental6Ano", "ATUFundamental7Ano", "ATUFundamental8Ano",
        "ATUFundamental9Ano", "HADFundamentalAnosFinais", "HADFundamental6Ano", "HADFundamental7Ano",
        "HADFundamental8Ano", "HADFundamental9Ano","IEDFundamentalAnosFinaisNivel1", "IEDFundamentalAnosFinaisNivel2",
        "IEDFundamentalAnosFinaisNivel3", "IEDFundamentalAnosFinaisNivel4", "IEDFundamentalAnosFinaisNivel5", 
        "IEDFundamentalAnosFinaisNivel6", "DSUFundamentalFinal","TDIFundamental6Ano", "TDIFundamental7Ano", "TDIFundamental8Ano",
        "TDIFundamental9Ano","TDIFundamentalAnosFinais","TDIMedioTotal"
]

# Pré-processamento dos dados para dfAnosIniciais
dfAnosIniciais.drop(colunas_a_serem_dropadas, axis=1, inplace=True)

# Pré-processamento dos dados para dfAnosFinaisTeste
dfAnosIniciaisTeste.drop(colunas_a_serem_dropadas, axis=1, inplace=True)

# Removendo linhas com valores nulos
dfAnosIniciais.fillna("NaN")
dfAnosIniciais[:] = np.nan_to_num(dfAnosIniciais)
dfAnosIniciaisTeste.fillna("NaN")
dfAnosIniciaisTeste[:] = np.nan_to_num(dfAnosIniciaisTeste)


# Análises

In [ ]:
# Análises

# analise 1: 2013 para prever 2013
# analise 2: 2015 para prever 2015
# analise 3: 2017 para prever 2017
# analise 4: 2019 para prever 2019

#lista de combinaçoes de anos:
combinacoes_anos = [
    [2013, 2013],
    [2015, 2015],
    [2017, 2017],
    [2019, 2019]
]

# Dicionario para armazenar os modelos de Regressao
modelos = {}

# Crie um dicionário para armazenar os coeficientes
coeficientes_por_analise = {}


# Loop pelas combinações de anos
for combo in combinacoes_anos:
    anos_treino = list(combo[:-1])
    ano_teste = combo[-1]

    # Filtrar os dados de treinamento e teste
    df_treino = dfAnosIniciais[dfAnosIniciais['Ano'].isin(anos_treino)]
    df_teste = dfAnosIniciaisTeste[dfAnosIniciaisTeste["Ano"] == ano_teste]

    # Separar as variáveis independentes e a variável alvo
    X_treino = df_treino.drop(["Ano", "anosIniciais"], axis=1)
    y_treino = df_treino["anosIniciais"]

    X_teste = df_teste.drop(["Ano", "anosIniciais"], axis=1)
    y_teste = df_teste["anosIniciais"]

    # Fazendo o train_test_split
    X_treino, X_teste, y_treino, y_teste = train_test_split(X_treino, y_treino, test_size=0.3, random_state=1)

    # Nomes das colunas antes da transformação
    nomes_colunas = X_treino.columns.tolist()

    # Quantile Transformer
    from sklearn.preprocessing import QuantileTransformer 
    qt = QuantileTransformer(output_distribution="normal")
    X_treino = qt.fit_transform(X_treino)
    X_teste = qt.transform(X_teste)

    # Criar o modelo de regressão linear
    modelo = LinearRegression()

    # Treinar o modelo
    modelo.fit(X_treino, y_treino)

    # Realizar previsões nos dados de teste
    y_pred = modelo.predict(X_teste)

    # coeficientes:
    coeficientes = modelo.coef_
    coeficientes = sorted(coeficientes, reverse=True)

    # Nomes das variáveis (nomes originais antes da transformação)
    nomes_variaveis = nomes_colunas

    # Organiza os coeficientes com nomes em 10 melhores e 10 piores
    melhores = [{"variavel": nomes_variaveis[i], "coeficiente": coeficientes[i]} for i in range(10)]
    piores = [{"variavel": nomes_variaveis[-i-1], "coeficiente": coeficientes[-i-1]} for i in range(10)]

    # Armazenar o modelo e resultados
    nome_analise = f"{combo[:-1]} prevendo {ano_teste}"
    modelos[nome_analise] = {
        "modelo": modelo,
        "resultados": {
            "MSE": metrics.mean_squared_error(y_teste, y_pred),
            "RMSE": np.sqrt(metrics.mean_squared_error(y_teste, y_pred)),
            "MAE": metrics.mean_absolute_error(y_teste, y_pred)
        }
    }

    # Armazenar os resultados no dicionário
    coeficientes_por_analise[nome_analise] = {"melhores": melhores, "piores": piores}

    # Armazenar o modelo e resultados
    modelos[nome_analise] = {
        "modelo": modelo,
        "resultados": {
            "MSE": metrics.mean_squared_error(y_teste, y_pred),
            "RMSE": np.sqrt(metrics.mean_squared_error(y_teste, y_pred)),
            "MAE": metrics.mean_absolute_error(y_teste, y_pred)
        }
    }

    # armazena os resultados no dicionário
    nome_analise = f"{combo[:-1]} prevendo {ano_teste}"
    coeficientes_por_analise[nome_analise] = {"melhores": melhores, "piores": piores}

# Imprimir os resultados
for nome_analise, resultados in modelos.items():
    print(nome_analise)
    print("Resultados:")
    for metrica, valor in resultados["resultados"].items():
        print(f"{metrica}: {valor}")
    print()

# Converter o dicionário de resultados em um DataFrame
df_resultados = pd.DataFrame({k: v["resultados"] for k, v in modelos.items()}).T

# Criar um novo arquivo Excel e adicionar o DataFrame de resultados a ele
wb = Workbook()
ws_resultados = wb.active
ws_resultados.title = "Regressao Linear Resultados Anos Iniciais Intra Ano"

# Formatação
font_title = Font(bold=True)
alignment_center = Alignment(horizontal="center")

# Adicionar o título para a planilha de resultados
ws_resultados["A1"] = "Análise"
ws_resultados["A1"].font = font_title
ws_resultados["A1"].alignment = alignment_center

# Adicionar as colunas do DataFrame de resultados
for row in dataframe_to_rows(df_resultados, index=True, header=True):
    ws_resultados.append(row)

# Adicionar uma nova planilha para os coeficientes
ws_coeficientes = wb.create_sheet(title="Coeficientes")

# Formatação para a planilha de coeficientes
font_title_coef = Font(bold=True)
alignment_center_coef = Alignment(horizontal="center")

# Adicionar o título da planilha de coeficientes
ws_coeficientes["A1"] = "Análise"
ws_coeficientes["A1"].font = font_title_coef
ws_coeficientes["A1"].alignment = alignment_center_coef
ws_coeficientes["B1"] = "Top 10 Melhores Coeficientes"
ws_coeficientes["B1"].font = font_title_coef
ws_coeficientes["B1"].alignment = alignment_center_coef
ws_coeficientes["D1"] = "Top 10 Piores Coeficientes"
ws_coeficientes["D1"].font = font_title_coef
ws_coeficientes["D1"].alignment = alignment_center_coef

# Adicionar as informações dos coeficientes às colunas da planilha de coeficientes
for nome_analise, resultados in coeficientes_por_analise.items():
    ws_coeficientes.append([nome_analise])
    for melhor, pior in zip(resultados["melhores"], resultados["piores"]):
        ws_coeficientes.append(["", melhor["variavel"], round(melhor["coeficiente"], 3), pior["variavel"], round(pior["coeficiente"], 3)])

# Salvar o arquivo
wb.save("D:\Bolsa-Iniciacao_Cientifica\TCE_Estagio\Semana_40\ResultadosAnosIniciais_IntraAno.xlsx")
